In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import toimage
from keras.datasets import cifar10
import keras.utils as Kutils
import keras.backend as K
from sklearn.metrics.pairwise import pairwise_distances

# 毎回乱数のseedを同じにしておく(実験の再現のため)
np.random.seed(120671)        

# import my functions
from triplet_generator import TripletGenerator, make_triplet_loss_func, bpr_triplet_loss

Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras.backend import set_session,tensorflow_backend
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)) #利用量に合わせて確保
config.gpu_options.per_process_gpu_memory_fraction = 0.49 # 上限を0.49に抑える
set_session(tf.Session(config=config))

In [3]:
result_dir = '../exp/'
log_dir = '../logs/cifar10_tripletloss-online'

In [4]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
nclasses = 10

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [5]:
# 入力画像の次元
img_rows, img_cols = 32, 32

# チャネル数（RGBなので3）
img_channels = 3

# CIFAR-10データをロード
# (nb_samples, nb_rows, nb_cols, nb_channel) = tf
nb_cols = nb_rows = 32
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# 画素値を0-1に変換
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

# クラスラベル（0-9）をone-hotエンコーディング形式に変換
y_train = Kutils.to_categorical(y_train, nclasses)
y_test = Kutils.to_categorical(y_test, nclasses)

# trainをtrainとvalidationに分割
def shuffle_arrays(arrays, random_seed=np.random.randint(100000)):
    for ar in arrays:        
        np.random.seed(random_seed)        
        np.random.shuffle(ar)
shuffle_arrays([X_train,y_train])

batch_size=480
n_train_samples = 45120
n_val_samples = 4800
X_val = X_train[-n_val_samples:]
y_val = y_train[-n_val_samples:]
X_train = X_train[:n_train_samples]
y_train = y_train[:n_train_samples]


In [6]:
from keras.models import Model
from keras.layers import Dense, Activation,Conv2D,MaxPooling2D,Dropout,Flatten,BatchNormalization, Input, Lambda
from keras import regularizers

# make a submodel
# オリジナルのcifar10用モデルからの変更点
# 1. BatchNormalizationをしてみる．
# 2. loss関数をtriplet lossにしてみる.
#temp = Conv2D(32, (3, 3), padding="same", input_shape=X_train.shape[1:]))
inputs = Input(shape=(32, 32, 3))
temp = Conv2D(32, (3, 3), padding="same",name="Conv1")(inputs)
temp = Activation('relu')(temp)

temp = Conv2D(32, (3, 3),name="Conv2")(temp)
temp = BatchNormalization(name="BN2")(temp)
temp = Activation('relu',name="Relu2")(temp)
temp = MaxPooling2D(pool_size=(2, 2),name="Pool2")(temp)
temp = Dropout(0.25,name="DO2")(temp)

temp = Conv2D(64, (3, 3), padding='same',name="Conv3")(temp)
temp = BatchNormalization(name="BN3")(temp)
temp = Activation('relu',name="Relu3")(temp)
temp = Conv2D(64, (3, 3),name="Conv4")(temp)
temp = BatchNormalization(name="BN4")(temp)
temp = Activation('relu',name="Relu4")(temp)
temp = MaxPooling2D(pool_size=(2, 2),name="Pool4")(temp)
temp = Dropout(0.25,name="DO4")(temp)

temp = Flatten(name="Flatten5")(temp)
temp = Dense(512,name="Dense5")(temp)
temp = Activation('relu',name="Relu5")(temp)
feature = Dropout(0.5,name="DO5")(temp)

# wrap up the layers into a single layer-like object
feature = Lambda(lambda x: K.l2_normalize(x,axis=-1),name="l2normalization")(temp)

_submodel = Model(inputs,feature,name="cifar10_convnet")


In [7]:
# prepare main inputs
samples = Input(shape=(32,32,3), name="samples")
submodel = _submodel(samples)

# add layers for prediction
linear_classifier = Dense(nclasses, name="linear_classifier")(submodel)
predictions = Activation('softmax',name="predictions")(linear_classifier)


In [8]:
from keras.preprocessing.image import ImageDataGenerator

model4triplet = Model(inputs=[samples], outputs=[submodel])

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
datagen.fit(X_train)


num_cats = len(np.unique(y_train, axis=0))
base_batch_size = 80
num_base_step = len(X_train) / base_batch_size
n_triplets_exp = ((base_batch_size/num_cats)**2)*num_cats

val_datagen = ImageDataGenerator()
val_gen= val_datagen.flow(X_val,y_val,batch_size=batch_size)


In [9]:
# make triplet loss functions
alpha=0.2 # param for triplet loss.
model4triplet.compile(loss=make_triplet_loss_func(alpha),optimizer='adam')
tri_gen = TripletGenerator(datagen.flow(X_train,y_train, batch_size=base_batch_size), model4triplet)


In [10]:
'''
# test code.
y= np.array([[0,1],[0,1],[0,1],[1,0],[1,0]])
X = np.random.rand(5,2)
#X = np.array([[1.0,0.5],[0.2,0.3],[0.4,0.1],[0.7,0.5],[0.8,0.5]])
triplets,triplets_y = tri_gen.get_triplets(X,y,embeddings=X)

test_feature = K.constant(triplets) 
test_y = K.constant(triplets_y)
#print(triplets)
#print(triplets_y)
#loss = tri_gen.triplet_loss(test_y,test_feature)


loss = tri_gen.triplet_loss(test_y,test_feature)
print(K.eval(loss))
loss = tri_gen.bpr_triplet_loss(test_y,test_feature)
print(K.eval(loss))
'''

'\n# test code.\ny= np.array([[0,1],[0,1],[0,1],[1,0],[1,0]])\nX = np.random.rand(5,2)\n#X = np.array([[1.0,0.5],[0.2,0.3],[0.4,0.1],[0.7,0.5],[0.8,0.5]])\ntriplets,triplets_y = tri_gen.get_triplets(X,y,embeddings=X)\n\ntest_feature = K.constant(triplets) \ntest_y = K.constant(triplets_y)\n#print(triplets)\n#print(triplets_y)\n#loss = tri_gen.triplet_loss(test_y,test_feature)\n\n\nloss = tri_gen.triplet_loss(test_y,test_feature)\nprint(K.eval(loss))\nloss = tri_gen.bpr_triplet_loss(test_y,test_feature)\nprint(K.eval(loss))\n'

In [11]:
import keras.callbacks

def train_triplet_loss(epochs, steps_per_epoch=int(num_base_step * (n_triplets_exp/batch_size)) / 10):
    model = Model(inputs=[samples], outputs=[submodel])
    model.compile(loss=make_triplet_loss_func(alpha),optimizer='adam')

    model.summary()

    fpath = "%s/weights_tripletloss_only.{epoch:02d}.hdf5'"%log_dir

    cp_cb = keras.callbacks.ModelCheckpoint(fpath, monitor='val_predictions_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=100)
    warnings.filterwarnings('ignore')

    return model.fit_generator(tri_gen.triplet_flow(batch_size),
                        steps_per_epoch=steps_per_epoch,
                        validation_data=val_gen,
                        validation_steps=10, 
                        epochs=epochs,
                        callbacks=[cp_cb],
                               )

def train_cross_entropy(epochs,steps_per_epoch=n_train_samples/batch_size):
    model = Model(inputs=[samples], outputs=[predictions])

    model.compile(loss={'predictions':'categorical_crossentropy'},
                  optimizer='adam', 
                  metrics={'predictions':'accuracy'})
    model.summary()

    # steps_per_epoch should be (number of training images total / batch_size) 
    # validation_steps should be (number of validation images total / batch_size) 
    fpath = "%s/weights_crossentropy_only.{epoch:02d}.hdf5'"%log_dir
    cp_cb = keras.callbacks.ModelCheckpoint(
        fpath,
        monitor='val_predictions_loss', 
        verbose=1, 
        save_best_only=False, save_weights_only=False, mode='auto', period=5)

    train_flow = datagen.flow(X_train[:45120],y_train[:45120], batch_size=batch_size)

    return model.fit_generator(train_flow,
                        steps_per_epoch=steps_per_epoch,
                        validation_data=val_gen,
                        validation_steps=10, 
                        epochs=epochs,
                        callbacks=[cp_cb],
                               )

def train_mix(epochs, loss_weights,  steps_per_epoch=int(num_base_step * (n_triplets_exp/batch_size)) / 10):
    class flow_wrapper:
        def __init__(self,baseflow):
            self.baseflow = baseflow
        def flow(self):
            while True:
                yield self._generator()
        def _generator(self):
            X,y = next(self.baseflow)
            return X,[y,y]

    model = Model(inputs=[samples], outputs=[predictions, submodel])
    model.compile(loss={'predictions':'categorical_crossentropy','cifar10_convnet':make_triplet_loss_func(alpha)},
                  optimizer='adam', loss_weights=loss_weights,
                  metrics={'predictions':'accuracy'})
    model.summary()

    fpath = "%s/weights_tripletloss_crossentropy.{epoch:02d}.hdf5'"%log_dir

    cp_cb = keras.callbacks.ModelCheckpoint(fpath, monitor='val_predictions_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=100)
    warnings.filterwarnings('ignore')
    return model.fit_generator(flow_wrapper(tri_gen.triplet_flow(batch_size)).flow(),
                        steps_per_epoch=steps_per_epoch,
                        validation_data=flow_wrapper(val_gen).flow(),
                        validation_steps=10, 
                        epochs=epochs,
                        callbacks=[cp_cb],
                               )
    


In [14]:
import multiprocessing
def eval_density(X,y, model4triplet):
    def _eval_density(X,y, model4triplet):
        embeddings = model4triplet.predict(X)
        dist_mat = pairwise_distances(embeddings, metric='sqeuclidean', n_jobs=multiprocessing.cpu_count())
        #print(dist_mat
        _sum = np.sum(dist_mat,axis=None)
        n_samples = len(X)
        return _sum/(n_samples*(n_samples-1))

    cats = np.unique(y, axis=0)
    for c in cats:
        c_samples = [np.all(a==b) for a,b in zip(y,[c]*len(y))]
        X_ = X[c_samples]
        y_ = y[c_samples]
        del c_samples
        avg_dist = _eval_density(X_,y_,model4triplet)
        print("cat: %02d avg. l2 dist: %0.5f"%(np.where(c)[0][0],avg_dist))
        


In [ ]:
train_cross_entropy(1)
eval_density(X_train,y_train,model4triplet)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
samples (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
cifar10_convnet (Model)      (None, 512)               1246368   
_________________________________________________________________
linear_classifier (Dense)    (None, 10)                5130      
_________________________________________________________________
predictions (Activation)     (None, 10)                0         
Total params: 1,251,498
Trainable params: 1,251,178
Non-trainable params: 320
_________________________________________________________________
Epoch 1/1
 7/94 [=>............................] - ETA: 445s - loss: 1.7226 - acc: 0.4220

In [ ]:
train_mix(1,[0.5,1.0])
eval_density(X_train,y_train,model4triplet)

In [ ]:
train_triplet_loss(1)
eval_density(X_train,y_train,model4triplet)